In [19]:
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt

In [15]:
data = pd.read_csv("./data/modify_other_train.csv",index_col=0)
print(data)

person = pd.read_csv("./data/personal_train.csv")
print(person)

numeric = pd.DataFrame()
categoric = pd.DataFrame()

for column in data:
    if data[column].dtype == np.float64 or data[column].dtype == np.int64:
        numeric[column] = data[column]
    else:
        categoric[column] = data[column]
        
print(numeric.dtypes)
print('\n')
print(categoric.dtypes)

                    name                                            address  \
0     Clarence Rodriguez        6931 Anthony Island\r\nTaylorland, NH 14540   
1            Manuel Boyd  50488 Timothy Knolls\r\nNorth Brittanyborough,...   
2         Charles Arthur  247 Johnson Roads Apt. 514\r\nPort Sarahton, W...   
3         Carl Dobransky         102 Taylor Trace\r\nNew Jennifer, RI 87597   
4         Thomas Oconnor                       USNV Skinner\r\nFPO AE 19822   
...                  ...                                                ...   
3978       Michael Gibbs  16431 Nelson Estate\r\nEast Samanthaberg, MN 6...   
3979      Ronald Nolasco          588 Timothy Field\r\nSteveville, TX 56369   
3980         Daniel Bell     78234 Emily Oval\r\nPort Michaelport, OR 24683   
3981      Joshua Goodwin                 PSC 4150, Box 2967\r\nAPO AA 04681   
3982      Howard Benally  060 Harris Overpass\r\nPort Justinborough, IL ...   

        race       marital-status          occupati

In [3]:
# Stary parser
# numeric = numeric.replace([' ?','??',' ','?',' ??',None],np.NaN,regex=True)
# categoric = categoric.replace({'^[? ]*$' : None},regex=True)

numeric = numeric.replace({ 0 : np.nan },regex=True)
categoric = categoric.replace({'^[? ]*$' : np.nan},regex=True)

categoric.info()
numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 11 columns):
name              3983 non-null object
address           3983 non-null object
race              3970 non-null object
marital-status    3965 non-null object
occupation        3731 non-null object
pregnant          3961 non-null object
relationship      3967 non-null object
education         3967 non-null object
income            3965 non-null object
native-country    3905 non-null object
workclass         3366 non-null object
dtypes: object(11)
memory usage: 373.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 14 columns):
education-num       3962 non-null float64
capital-gain        3970 non-null float64
fnlwgt              3966 non-null float64
class               3966 non-null float64
hours-per-week      3968 non-null float64
capital-loss        3969 non-null float64
mean_glucose        3983 non-null float64
std_glucose         

In [4]:
numeric['name'] = data['name']
print(numeric.dtypes)
print(numeric)
print('\n')
print(categoric.dtypes)
print(categoric)

education-num       float64
capital-gain        float64
fnlwgt              float64
class               float64
hours-per-week      float64
capital-loss        float64
mean_glucose        float64
std_glucose         float64
kurtosis_glucose    float64
skewness_glucose    float64
mean_oxygen         float64
std_oxygen          float64
kurtosis_oxygen     float64
skewness_oxygen     float64
name                 object
dtype: object
      education-num  capital-gain    fnlwgt  class  hours-per-week  \
0               9.0           0.0  183279.0    0.0            40.0   
1              10.0           0.0  130724.0    1.0            25.0   
2              10.0           0.0  180284.0    0.0            50.0   
3              13.0           0.0  126060.0    0.0            40.0   
4               7.0           0.0  274657.0    1.0            40.0   
...             ...           ...       ...    ...             ...   
3978            9.0           0.0   29814.0    0.0            50.0   
3979  

In [5]:
numeric = numeric.groupby('name').aggregate('mean')

categoric = categoric.groupby('name').aggregate('first')

other = categoric.merge(numeric, on=['name'])
other

,address,race,marital-status,occupation,pregnant,relationship,education,income,native-country,workclass,...,hours-per-week,capital-loss,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen
name,,,,,,,,,,,,,,,,,,,,,
Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",Asian-Pac-Islander,Married-civ-spouse,Craft-repair,f,Husband,Some-college,>50K,United-States,Private,...,40.0,0.0,111.554688,52.480772,0.390191,-0.176512,0.530936,10.232655,21.400696,479.743759
Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",White,Divorced,Machine-op-inspct,F,Unmarried,HS-grad,<=50K,United-States,Private,...,40.0,0.0,110.257812,56.334554,0.280454,-0.260235,2.142140,13.440810,10.702973,149.848180
Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,White,Married-civ-spouse,Sales,FALSE,Husband,Bachelors,<=50K,United-States,Private,...,40.0,0.0,128.187500,49.815088,-0.016114,-0.322560,1.683946,12.169941,12.270117,198.354692
Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",White,Divorced,Craft-repair,f,Not-in-family,HS-grad,<=50K,United-States,Private,...,40.0,0.0,101.382812,51.082672,0.361766,-0.168472,3.933110,21.785797,8.375144,80.188004
Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",White,Never-married,Sales,f,Own-child,Some-college,<=50K,United-States,Self-emp-inc,...,35.0,0.0,75.640625,34.337192,1.073438,4.466385,6.118729,28.610537,5.188312,28.565339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Provines,"7993 Sharon Roads Suite 311\r\nPort Daniel, GA...",White,Never-married,Craft-repair,f,Own-child,11th,<=50K,Mexico,Self-emp-not-inc,...,50.0,0.0,127.796875,54.659508,-0.017076,-0.424223,1.439799,15.503820,12.930222,178.171016
Zachary Grandfield,"77606 Maxwell Rest Apt. 485\r\nPort Jennifer, ...",White,Married-civ-spouse,Exec-managerial,f,Husband,Bachelors,>50K,United-States,Private,...,50.0,1977.0,91.734375,47.215842,0.605275,0.604172,9.842809,40.384259,4.088567,15.468653
Zachary Miller,"96654 Marc Stream\r\nEast Cassandra, VT 84597",White,Never-married,Exec-managerial,f,Not-in-family,12th,>50K,United-States,State-gov,...,50.0,0.0,141.757812,91.206475,-0.056413,-1.755332,55.243311,75.793948,1.047847,-0.255939


In [6]:
data_merged = other.merge(person, on=['name', 'address'])
data_merged


,name,address,race,marital-status,occupation,pregnant,relationship,education,income,native-country,...,kurtosis_glucose,skewness_glucose,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen,Unnamed: 0,age,sex,date_of_birth
0,Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",Asian-Pac-Islander,Married-civ-spouse,Craft-repair,f,Husband,Some-college,>50K,United-States,...,0.390191,-0.176512,0.530936,10.232655,21.400696,479.743759,3657,41.0,Male,1978-08-08
1,Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",White,Divorced,Machine-op-inspct,F,Unmarried,HS-grad,<=50K,United-States,...,0.280454,-0.260235,2.142140,13.440810,10.702973,149.848180,1751,38.0,Female,1981-06-22
2,Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,White,Married-civ-spouse,Sales,FALSE,Husband,Bachelors,<=50K,United-States,...,-0.016114,-0.322560,1.683946,12.169941,12.270117,198.354692,546,36.0,Male,1983-09-22
3,Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",White,Divorced,Craft-repair,f,Not-in-family,HS-grad,<=50K,United-States,...,0.361766,-0.168472,3.933110,21.785797,8.375144,80.188004,1616,NaN,Male,1972-07-14 00 00 00
4,Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",White,Never-married,Sales,f,Own-child,Some-college,<=50K,United-States,...,1.073438,4.466385,6.118729,28.610537,5.188312,28.565339,1345,27.0,Female,1991-12-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,Young Provines,"7993 Sharon Roads Suite 311\r\nPort Daniel, GA...",White,Never-married,Craft-repair,f,Own-child,11th,<=50K,Mexico,...,-0.017076,-0.424223,1.439799,15.503820,12.930222,178.171016,3475,NaN,Male,1961-07-28
3929,Zachary Grandfield,"77606 Maxwell Rest Apt. 485\r\nPort Jennifer, ...",White,Married-civ-spouse,Exec-managerial,f,Husband,Bachelors,>50K,United-States,...,0.605275,0.604172,9.842809,40.384259,4.088567,15.468653,2644,74.0,Male,45-09-15
3930,Zachary Miller,"96654 Marc Stream\r\nEast Cassandra, VT 84597",White,Never-married,Exec-managerial,f,Not-in-family,12th,>50K,United-States,...,-0.056413,-1.755332,55.243311,75.793948,1.047847,-0.255939,3821,71.0,Male,11/07/1948
3931,Zachary Seltzer,"38371 Crystal Stravenue\r\nPerkinsmouth, DE 20764",Black,Never-married,Exec-managerial,f,Not-in-family,Some-college,<=50K,United-States,...,0.529680,0.149780,4.782609,26.230065,6.377345,42.574512,2188,40.0,Male,1978-11-02


In [8]:
for i in data_merged:
    if (data_merged[i].dtype == np.float64):
        imp = Imputer(missing_values = np.NaN, strategy = 'mean')
        data_merged[i] = imp.fit_transform(data_merged[[i]])
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3932
Data columns (total 29 columns):
name                3933 non-null object
address             3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3703 non-null object
pregnant            3931 non-null object
relationship        3932 non-null object
education           3933 non-null object
income              3932 non-null object
native-country      3872 non-null object
workclass           3341 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3933 non-null float64
class               3933 non-null float64
hours-per-week      3933 non-null float64
capital-loss        3933 non-null float64
mean_glucose        3933 non-null float64
std_glucose         3933 non-null float64
kurtosis_glucose    3933 non-null float64
skewness_glucose    3933 non-null float64
mean_oxygen         39

In [9]:
data_merged['pregnant'].value_counts()

f        3466
F         200
FALSE     182
t          72
TRUE        6
T           5
Name: pregnant, dtype: int64

In [10]:
data_merged = data_merged.replace(['FALSE','F'],'f',regex=True)
data_merged = data_merged.replace(['TRUE','T'],'t',regex=True)
data_merged['pregnant'].value_counts()

f    3848
t      83
Name: pregnant, dtype: int64

In [11]:
data_merged['income'].value_counts()

 <=50K    2646
 >50K      893
<=50K      314
>50K        79
Name: income, dtype: int64

In [12]:
data_merged = data_merged.replace([' <=50K'],'<=50K',regex=True)
data_merged = data_merged.replace([' >50K'],'>50K',regex=True)
data_merged['income'].value_counts()

<=50K    2960
>50K      972
Name: income, dtype: int64

In [13]:
data_merged['relationship'].value_counts()

 Husband           1596
 Not-in-family      904
 Own-child          564
 Unmarried          391
 Wife               192
 Not_in_family      114
 Other-relative     110
 Own_child           49
 Other_relative      12
Name: relationship, dtype: int64

In [14]:
data_merged = data_merged.replace([' Other-relative'],' Other_relative',regex=True)
data_merged = data_merged.replace([' Not-in-family'],' Not_in_family',regex=True)
data_merged = data_merged.replace([' Own-child'],' Own_child',regex=True)
data_merged['relationship'].value_counts()

 Husband           1596
 Not_in_family     1018
 Own_child          613
 Unmarried          391
 Wife               192
 Other_relative     122
Name: relationship, dtype: int64

In [18]:
data_merged['date_of_birth'].value_counts()

1974-04-25    3
1967-04-29    3
1972-03-31    3
1964-03-10    3
1959-10-14    3
             ..
1967-03-27    1
1975-07-18    1
1986/05/30    1
1980-11-16    1
1989-02-16    1
Name: date_of_birth, Length: 3703, dtype: int64

In [21]:
for i, row in data_merged.iterrows():
        
    date = parser.parse(data_merged.iloc[i]['date_of_birth'].split(' ')[0])
    day = date.day
    month = date.month
    year = date.year
    
    if (year > 2019):
        year = year - 100

    data_merged.at[i,'date_of_birth'] = str(day) + "/" + str(month) + "/" + str(year)
            
    if ((pd.isna(row['age'])) | (row['age'] > 115) | (row['age'] < 0)):
        data_merged.at[i,'age'] = 2019 - year
        
data_merged

,name,address,race,marital-status,occupation,pregnant,relationship,education,income,native-country,...,kurtosis_glucose,skewness_glucose,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen,Unnamed: 0,age,sex,date_of_birth
0,Aaron Davis,"379 Rodriguez Inlet\r\nfisherbury, MD 83114",Asian-Pac-Islander,Married-civ-spouse,Craft-repair,f,Husband,Some-college,>50K,United-States,...,0.390191,-0.176512,0.530936,10.232655,21.400696,479.743759,3657,41.0,Male,8/8/1978
1,Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",White,Divorced,Machine-op-inspct,f,Unmarried,HS-grad,<=50K,United-States,...,0.280454,-0.260235,2.142140,13.440810,10.702973,149.848180,1751,38.0,female,22/6/1981
2,Aaron Gann,59685 thomas Extension Apt. 142\r\nfieldsborou...,White,Married-civ-spouse,Sales,f,Husband,Bachelors,<=50K,United-States,...,-0.016114,-0.322560,1.683946,12.169941,12.270117,198.354692,546,36.0,Male,22/9/1983
3,Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",White,Divorced,Craft-repair,f,Not_in_family,HS-grad,<=50K,United-States,...,0.361766,-0.168472,3.933110,21.785797,8.375144,80.188004,1616,47.0,Male,14/7/1972
4,Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",White,Never-married,Sales,f,Own_child,Some-college,<=50K,United-States,...,1.073438,4.466385,6.118729,28.610537,5.188312,28.565339,1345,27.0,female,24/12/1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,Young Provines,"7993 Sharon Roads Suite 311\r\nPort Daniel, GA...",White,Never-married,Craft-repair,f,Own_child,11th,<=50K,Mexico,...,-0.017076,-0.424223,1.439799,15.503820,12.930222,178.171016,3475,58.0,Male,28/7/1961
3929,Zachary Grandfield,"77606 Maxwell Rest Apt. 485\r\nPort Jennifer, ...",White,Married-civ-spouse,Exec-managerial,f,Husband,Bachelors,>50K,United-States,...,0.605275,0.604172,9.842809,40.384259,4.088567,15.468653,2644,74.0,Male,15/9/1945
3930,Zachary Miller,"96654 Marc Stream\r\nEast Cassandra, Vt 84597",White,Never-married,Exec-managerial,f,Not_in_family,12th,>50K,United-States,...,-0.056413,-1.755332,55.243311,75.793948,1.047847,-0.255939,3821,71.0,Male,11/7/1948
3931,Zachary Seltzer,"38371 Crystal Stravenue\r\nPerkinsmouth, DE 20764",Black,Never-married,Exec-managerial,f,Not_in_family,Some-college,<=50K,United-States,...,0.529680,0.149780,4.782609,26.230065,6.377345,42.574512,2188,40.0,Male,11/2/1978


In [ ]:
numeric = data_merged.select_dtypes(include=['float64'])

# Remove NULL value for numeric columns except mean_glucose
for item in data_merged:
    if (data_merged[item].dtype == np.float64): #and i != 'mean_glucose' :
        imp = Imputer(missing_values=np.NaN, strategy='mean')
        data_merged[item] = imp.fit_transform(data_merged[[item]])

In [ ]:
categoric =  data_merged.select_dtypes(include=['object'])

# Remove NULL value for categorical columns
for item in data_merged:
    if (data_merged[item].dtype == np.object):
        imp = Imputer(missing_values=np.nan, strategy='most_frequent')
        data_merged[item] = imp.fit_transform(data_merged[[item]])

In [ ]:
# KNN regression to know how many groups make

objectsTrain = numeric[numeric.isna().any(axis=1) == False]
objectsPredict = numeric[numeric.isna().any(axis=1) == True]
Xtrain,Xtest,Ytrain,Ytest = split(objectsTrain['skewness_glucose'],objectsTrain['mean_glucose'],test_size=0.3)

scores = []
Xtrain = Xtrain.values.reshape(-1,1)
#Ytrain = Ytrain.values.reshape(-1,1)
Xtest = Xtest.values.reshape(-1,1)
#Ytest = Ytest.values.reshape(-1,1)
for k in range(1,300):
    model = neigh(n_neighbors = k)    
    model.fit(Xtrain,Ytrain)
    Ypred = model.predict(Xtest)
    scores.append(sqrt(metrics.mean_squared_error(Ytest,Ypred)))

In [ ]:
print(scores.index(min(scores)),' : ', min(scores))
curve = pd.DataFrame(scores)
curve.plot()

In [ ]:
## Real prediction on mean_glucose with skewness_glucose and use k group
## !!!IMPORTANT!!! need to add into datas

# Xpred = objectsPredict['skewness_glucose'].values.reshape(-1,1)
# Ypred = objectsPredict['mean_glucose']

# Xtrain = objectsTrain['skewness_glucose'].values.reshape(-1,1)
# Ytrain = objectsTrain['mean_glucose'].values.reshape(-1,1)
# k = 98
# model = neigh(n_neighbors = k)    
# model.fit(Xtrain,Ytrain)

# Ypred = model.predict(Xpred)

In [ ]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

# Get only float64 columns from datas
numeric = data_merged.select_dtypes(include=['float64'])

In [ ]:
# Print all distplot from numeric datas

for i in numeric:
    plt.figure(i)
    sns.distplot(numeric[i])

In [ ]:
## Replace all numeric outliers
## !!!IMPORTANT!!! need add another methods to replace outliers like transform value by using sqrt or logaritmic function

for i in numeric:
    median = numeric[i].median()
    std = numeric[i].std()  
    outliers = identify_outliers(numeric[i])
    numeric[i] = numeric[i].replace(outliers.index,np.nan)
    numeric[i].fillna(median,inplace=True)

In [ ]:
##
## !!!IMPORTANT!!! need some persistance on models(replace NULL value, replace Outliers)
